In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from randomtimestamp import randomtimestamp, random_date, random_time
from datetime import datetime
from datetime import timedelta
# Set figure size to (28,14)
plt.rcParams['figure.figsize'] = (28, 14)


In [2]:
#5 Calculate the total number of customers present in the supermarket over time.

def question_05(df):
    df = pd.DataFrame(df.groupby('timestamp').count()['customer_no'])
    sns.lineplot(data=df)

In [3]:
def question_04(df):
    df.reset_index(inplace=True)
    dict = {}
    # df_first = pd.DataFrame(columns=df.columns)
    # df_second = pd.DataFrame(columns=df.columns)
    for i in range(1,df['customer_no'].max(),1):
        #print(i)
        print(df[df['customer_no'] == i].iloc[0, 0])
        a = df[df['customer_no'] == i].iloc[-1, 0] - \
            df[df['customer_no'] == i].iloc[0, 0]
        dict[i] = a
        # df_first = df_first.append(a)
        # #print(df_first.head())
        # b = df[df['customer_no'] == i].iloc[1:]
        # df_second = df_second.append(b)
    # fig, axs = plt.subplots(2)
    # sns.histplot(df_first, x="location", ax=axs[0])
    # sns.histplot(df_second, x='location', ax=axs[1])
    return dict

In [4]:
# dict = question_06(df_0)

In [5]:
# Create customer_id
def create_id(df, letters):
    df['customer_id'] = letters + '_' + df['customer_no'].astype(str)

In [6]:
# Function for sorting
def sort_df(df):
    df.sort_index(inplace=True)
    df.sort_values('customer_no', kind='stable', inplace=True)
    return df


In [7]:
# Function for filling
def filling_steps(df):
    df_n = df.groupby('customer_no').resample('1T').ffill()
    df_l = df_n[['customer_no', 'location']].set_index('customer_no')
    df_l.reset_index(inplace=True)
    return df_l


In [8]:
# Function for adding checkouts to customers without
def checkout(df):
    for i in range(1, df['customer_no'].max()+1, 1):
        if df[df['customer_no'] == i].iloc[-1, 1] != 'checkout':
            a = df[df['customer_no'] == i].iloc[0,0]
            a_df = pd.DataFrame({"customer_no":[a],'location':'checkout'})
            df = df.append(a_df, ignore_index=True)
    return sort_df(df)

In [9]:
# create Markov columns
def markov_columns(df):
    df["before"] = df['location']
    df['after'] = df['location'].shift(-1)
    df.loc[df['before'] == 'checkout', 'after'] = 'checkout'
    df.set_index('customer_id', inplace=True)
    df.drop(columns=['customer_no','location'], inplace=True)
    return df


In [10]:
df_0 = pd.read_csv('data/monday.csv', parse_dates=True, index_col=0, sep=';')
df_1 = pd.read_csv('data/tuesday.csv', parse_dates=True, index_col=0, sep=';')
df_2 = pd.read_csv('data/wednesday.csv', parse_dates=True, index_col=0, sep=';')
df_3 = pd.read_csv('data/thursday.csv', parse_dates=True, index_col=0, sep=';')
df_4 = pd.read_csv('data/friday.csv', parse_dates=True, index_col=0, sep=';')


In [11]:
df_lst = [df_0, df_1, df_2, df_3, df_4]
days = ['MO','TU','WE','TH','FR']

In [12]:
for i in range(len(df_lst)):
    df_lst[i] = sort_df(df_lst[i])  # df_0 = sort_df(df_0)
    df_lst[i] = filling_steps(df_lst[i])
    df_lst[i] = checkout(df_lst[i])
    create_id(df_lst[i],days[i])   # create_id(df_0, 'MO')


In [13]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4884 entries, 2019-09-02 07:03:00 to 2019-09-02 21:50:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  4884 non-null   int64 
 1   location     4884 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.5+ KB


In [14]:
df_0.head(20)

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:05:00,1,checkout
2019-09-02 07:03:00,2,dairy
2019-09-02 07:06:00,2,checkout
2019-09-02 07:04:00,3,dairy
2019-09-02 07:06:00,3,checkout
2019-09-02 07:04:00,4,dairy
2019-09-02 07:08:00,4,checkout
2019-09-02 07:04:00,5,spices


In [15]:
df_lst[1].tail(10)

,customer_no,location,customer_id
10114,1420,drinks,TU_1420
10115,1420,drinks,TU_1420
10116,1420,drinks,TU_1420
10117,1420,checkout,TU_1420
10118,1421,fruit,TU_1421
10119,1421,drinks,TU_1421
10120,1421,checkout,TU_1421
10121,1422,spices,TU_1422
10122,1422,dairy,TU_1422
10124,1422,checkout,TU_1422


In [16]:
#we create a list with every single df in order to concatenate them together
df = pd.concat(df_lst)
df.head(50)


,customer_no,location,customer_id
0,1,dairy,MO_1
1,1,dairy,MO_1
2,1,checkout,MO_1
3,2,dairy,MO_2
4,2,dairy,MO_2
5,2,dairy,MO_2
6,2,checkout,MO_2
7,3,dairy,MO_3
8,3,dairy,MO_3
9,3,checkout,MO_3


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53931 entries, 0 to 11075
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  53931 non-null  int64 
 1   location     53931 non-null  object
 2   customer_id  53931 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [18]:
df.tail(30)

,customer_no,location,customer_id
11044,1500,fruit,FR_1500
11045,1500,fruit,FR_1500
11046,1500,dairy,FR_1500
11070,1500,checkout,FR_1500
11047,1501,fruit,FR_1501
11048,1501,fruit,FR_1501
11049,1501,checkout,FR_1501
11050,1502,fruit,FR_1502
11051,1502,checkout,FR_1502
11052,1503,dairy,FR_1503


In [19]:
# General eda 
df['customer_id'].value_counts()

TH_1047    53
FR_102     53
WE_710     52
MO_666     52
FR_1238    50
           ..
TH_121      2
MO_1182     2
MO_175      2
MO_1445     2
FR_911      2
Name: customer_id, Length: 7445, dtype: int64

In [20]:
df_mar = markov_columns(df)
df_mar.head()


,before,after
customer_id,,
MO_1,dairy,dairy
MO_1,dairy,checkout
MO_1,checkout,checkout
MO_2,dairy,dairy
MO_2,dairy,dairy


In [21]:
df_mar.tail(50)

,before,after
customer_id,,
FR_1497,dairy,dairy
FR_1497,dairy,dairy
FR_1497,dairy,checkout
FR_1497,checkout,checkout
FR_1498,fruit,fruit
FR_1498,fruit,checkout
FR_1498,checkout,checkout
FR_1499,dairy,dairy
FR_1499,dairy,dairy


In [22]:
df_mar.to_csv('df_mar.csv')

In [23]:
P = pd.crosstab(df_mar['before'], df_mar['after'], normalize=0)
P

after,checkout,dairy,drinks,fruit,spices
before,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.103466,0.736750,0.058569,0.049848,0.051367
drinks,0.215710,0.010899,0.598499,0.087909,0.086983
fruit,0.201605,0.095924,0.054847,0.596947,0.050677
spices,0.150685,0.193214,0.163109,0.090953,0.402039


In [24]:
states = ['checkout','dairy','drinks',"fruit",'spices']

In [25]:
random.choice(states)


'checkout'

In [26]:
# convery the transition matrix to a dictionary that maps state to a list of probabilities
# order such that the first value is transition probability to cold and the second value
# is transition probability to hot

probs = P.to_dict(orient='index')
#probs

In [27]:
# convert internal values, which is a dict to a list
for key in probs.keys():
    probs[key] = list(probs[key].values())



In [28]:
#probs

In [ ]:
# class Customer1:
#     """
#     a single customer that moves through the supermarket
#     in a MCMC simulation
#     """

#     def __init__(self, id, state, transition_probs, budget=100):
#         self.id = id
#         self.state = state
#         self.transition_probs = transition_probs
#         self.budget = budget

#     def __repr__(self):
#         return f'<Customer_no {self.id} in {self.state}>'

#     def next_state(self):
#         '''
#         Propagates the customer to the next state.
#         Returns nothing.
#         '''
#         #self.state = random.choice(['spices', 'drinks', 'fruit'])
#         self.state = random.choices(states, weights=self.transition_probs[self.state])[0]
#         return self.state

#     def is_active(self):
#         """Returns True if the customer has not reached the checkout yet."""
#         return self.state != 'checkout'



In [ ]:
# bagigio = Customer1('bagigio','dairy',probs)

In [ ]:
# lst_customers = [bagigio]

In [ ]:
# for i in lst_customers:
#     i = 0   
#     min_time = random_time(text=True, pattern='%H:%M:%S')
#     # create datetime object from timestamp string
#     given_time = datetime.strptime(min_time, "%H:%M:%S")
#     print(type(given_time))
#     print(f"{given_time.time()} -- {bagigio.id} is in {bagigio.state} section")
#     print(type(given_time))
#     while bagigio.is_active():    
#         i+=1
#         print(f"{(given_time + timedelta(minutes = i)).time()} -- {bagigio.id} is in {bagigio.next_state()} section")


In [29]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """

    def __init__(self, id, state, budget=100):
        self.id = id
        self.state = state
        self.budget = budget

    def __repr__(self):
        return f'<Customer_no {self.id} in {self.state}>'

    def is_active(self):
        """Returns True if the customer has not reached the checkout yet."""
        return self.state != 'checkout'


In [113]:
"""
Start with this to implement the supermarket simulator.
"""


class Supermarket:
    """manages multiple Customer instances that are currently in the market.
    """

    def __init__(self, opening, closing):
        ### ADD OPENING AND CLOSING HOURS
        # a list of Customer objects
        self.dict_csv = {'timestamp':[],'customer_no':[],'location':[]}
        self.customers = []
        self.customers_timers = []
        self.time = (datetime.strptime(opening, "%H:%M"))
        self.end_time = (datetime.strptime(closing, "%H:%M"))

    def __repr__(self):
        return 'something about the supermarket'

    def get_time(self):
        """current time in HH:MM format,
        """
        return f"It's {self.time.time()}"
        # ??? just get back the current time

    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        df = pd.DataFrame(self.dict_csv)
        df.to_csv("supermarket.csv")
        return df


        # return self.d_customers_no, self.d_timestamp, self.d_location

    def next_minute(self):
        """propagates all customers to the next state.
        """
        minute = 0
        #if str(self.time.time()) == str((self.end_time - timedelta(minutes=5)).time()):


        while str(self.time.time()) != str(self.end_time.time()):
            self.time = (self.time + timedelta(minutes=minute))
            #print(str(self.time.time())[:-3])
            
            print()
            for i in range(self.number_clients):
                if self.customers_timers[i] == str(self.time.time())[:-3]:
                    self.dict_csv['timestamp'].append(self.customers_timers[i])
                    self.dict_csv['customer_no'].append(self.customers[i].id)
                    self.dict_csv['location'].append(self.customers[i].state)
                    internal_min = 0
                    while self.customers[i].is_active():
                        print(internal_min)
                        self.customers[i].state = random.choices(
                            states, weights=probs[self.customers[i].state])[0]
                        internal_min += 1
                        self.dict_csv['timestamp'].append(str((self.time + timedelta(minutes=internal_min)).time())[:-3])
                        self.dict_csv['customer_no'].append(self.customers[i].id)
                        self.dict_csv['location'].append(self.customers[i].state)

                        
                else:
                    continue
                    

            
            minute = 1


    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.number_clients = random.randint(100, 200)
        for i in range((self.number_clients)):
            self.customers.append(Customer(i, random.choice(states[1:])))
            self.customers_timers.append(random_time(start=self.time.time(), end=self.end_time.time(), text=True, pattern="%H:%M"))

        



    # def remove_exitsting_customers(self):
    #     """removes every customer that is not active any more.
    #     """
    #     for i in self.customers:
    #         if i.state == 'checkout':
    #             #self.customers.remove(i)
    #             self.customers.pop(i)
    #             self.customers_timers.pop(i)
        



In [114]:
"""
Start with this to implement the supermarket simulator.
"""


class Supermarket_01:
    """manages multiple Customer instances that are currently in the market.
    """

    def __init__(self, opening, closing):
        ### ADD OPENING AND CLOSING HOURS
        # a list of Customer objects
        self.dict_csv = {'timestamp':[],'customer_no':[],'location':[]}
        self.customers = []
        #self.customers_timers = []
        self.time = (datetime.strptime(opening, "%H:%M"))
        self.end_time = (datetime.strptime(closing, "%H:%M"))

    def __repr__(self):
        return 'something about the supermarket'

    def get_time(self):
        """current time in HH:MM format,
        """
        return f"It's {self.time.time()}"
        # ??? just get back the current time

    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        df = pd.DataFrame(self.dict_csv, index_col=0, parse_dates=True)
        df.to_csv("supermarket_01.csv")
        return df


        # return self.d_customers_no, self.d_timestamp, self.d_location

    def next_minute(self):
        """propagates all customers to the next state.
        """
        minute = 0
        #if str(self.time.time()) == str((self.end_time - timedelta(minutes=5)).time()):
        #active_lst = []
        
        while str(self.time.time()) != str(self.end_time.time()):
            self.time = (self.time + timedelta(minutes=minute))
                
            r = random.randint(0, self.number_clients)

            if self.customers[r].is_active():
                self.dict_csv['timestamp'].append(str(self.time.time()))
                self.dict_csv['customer_no'].append(self.customers[r].id)
                self.dict_csv['location'].append(self.customers[r].state)
                print(
                    f'{str(self.time.time())} customer_{self.customers[r].id} in {self.customers[r].state}')
                self.customers[r].state = random.choices(
                    states, weights=probs[self.customers[r].state])[0]


            # if self.customers[r] not in active_lst:
            #     active_lst.append(self.customers[r])
                
            #     for c in active_lst:
            #         if c.is_active():
            #             self.dict_csv['timestamp'].append(
            #                 str(self.time.time()))
            #             self.dict_csv['customer_no'].append(c.id)
            #             self.dict_csv['location'].append(c.state)
            #             print(f'{str(self.time.time())} customer_{c.id} in {c.state}')
            #             c.state = random.choices(states, weights=probs[c.state])[0]

            # elif self.customers[r] in active_lst:

            #     for c in active_lst:
            #         if c.is_active():
            #             self.dict_csv['timestamp'].append(
            #                 str(self.time.time()))
            #             self.dict_csv['customer_no'].append(c.id)
            #             self.dict_csv['location'].append(c.state)
            #             print(
            #                 f'{str(self.time.time())} customer_{c.id} in {c.state}')
            #             c.state = random.choices(
            #                 states, weights=probs[c.state])[0]
            # else: continue

            minute = 1
                


    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.number_clients = random.randint(1, 4)
        for i in range((self.number_clients)):
            self.customers.append(Customer(i, random.choice(states[1:])))
        
        print(self.customers)


    # def remove_exitsting_customers(self):
    #     """removes every customer that is not active any more.
    #     """
    #     for i in self.customers:
    #         if i.state == 'checkout':
    #             #self.customers.remove(i)
    #             self.customers.pop(i)
    #             self.customers_timers.pop(i)
        



In [110]:
s = Supermarket_01('07:00','22:00')

In [111]:
s.add_new_customers()

In [112]:
s.next_minute()

07:00:00 customer_0 in fruit


IndexError: list index out of range

In [58]:
df = s.print_customers()

In [59]:
df.head(40)

,timestamp,customer_no,location
0,07:00,122,dairy
1,07:01,122,dairy
2,07:02,122,dairy
3,07:03,122,dairy
4,07:04,122,dairy
5,07:05,122,dairy
6,07:06,122,drinks
7,07:07,122,drinks
8,07:08,122,drinks
9,07:09,122,checkout


In [60]:
df.shape

(1324, 3)

In [ ]:
print(k)
